In [26]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import joblib
import imodelsx.process_results
import sys
from sklearn.metrics import accuracy_score

# def load_csvs(path):

#     X_train = pd.read_csv(f'{path}/X_trainval.csv', index_col=0)
#     X_train_hat = pd.read_csv(f'{path}/X_trainval_hat.csv', index_col=0)
#     X_test = pd.read_csv(f'{path}/X_test.csv', index_col=0)
#     X_test_hat = pd.read_csv(f'{path}/X_test_hat.csv', index_col=0)
#     y_train = pd.read_csv(f'{path}/y_trainval.csv', index_col=0)
#     y_train_hat = pd.read_csv(f'{path}/y_trainval_hat.csv', index_col=0)
#     y_test = pd.read_csv(f'{path}/y_test.csv', index_col=0)
#     y_test_hat = pd.read_csv(f'{path}/y_test_hat.csv', index_col=0)

#     return X_train, X_train_hat, X_test, X_test_hat, y_train, y_train_hat, y_test, y_test_hat

sys.path.append('../experiments/')
results = []

results_dir = f'../results/cub_distillation_edit'
experiment_filename = '../experiments/cub_distillation_edit.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

# get the breakdown of data in these groups
r.groupby(cols_varied).size()

100%|██████████████████████████████████████████| 40/40 [00:00<00:00, 138.47it/s]


experiment varied these params: ['model_name', 'X_type', 'thresh', 'max_rules', 'max_trees', 'max_depth', 'pre_max_features', 'post_max_features', 'concepts_to_edit']


model_name           X_type   thresh  max_rules  max_trees  max_depth  pre_max_features  post_max_features  concepts_to_edit              
FIGSRegressor        binary   0.00    60         60         3          1.0               30.0               50,89,22,52,78,82,83,100,86,20    1
                                                            4          1.0               30.0               50,89,22,52,78,82,83,100,86,20    1
                                                 75         3          1.0               30.0               50,89,22,52,78,82,83,100,86,20    1
                                                            4          1.0               30.0               50,89,22,52,78,82,83,100,86,20    1
                                      90         60         3          1.0               30.0               50,89,22,52,78,82,83,100,86,20    1
                                                            4          1.0               30.0               50,89,22,52,78,82,83,100,86,20   

In [27]:
r.columns

Index(['task_type', 'model_name', 'X_type', 'thresh', 'Y_type', 'save_dir',
       'max_rules', 'max_trees', 'max_depth', 'pre_interaction',
       'pre_max_features', 'post_interaction', 'post_max_features', 'mo',
       'device', 'concepts_to_edit', 'use_cache', 'save_dir_unique',
       'cbm_true_seed1_accuracy_trainval', 'cbm_true_seed1_accuracy_test',
       'distiller_true_seed1_accuracy_trainval',
       'distiller_true_seed1_accuracy_test',
       'distiller_cbm_seed1_accuracy_trainval',
       'distiller_cbm_seed1_accuracy_test', '%_correct_seed1_overlap_trainval',
       '%_correct_seed1_overlap_test',
       'edited_cbm_true_seed1_accuracy_trainval',
       'edited_cbm_true_seed1_accuracy_test',
       'edited_distiller_true_seed1_accuracy_trainval',
       'edited_distiller_true_seed1_accuracy_test',
       'edited_distiller_cbm_seed1_accuracy_trainval',
       'edited_distiller_cbm_seed1_accuracy_test',
       'edited_%_correct_seed1_overlap_trainval',
       'edited_%_cor

In [28]:
#performance
r['mean_cbm_true_accuracy_trainval'] = np.mean([r[f'cbm_true_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['std_cbm_true_accuracy_trainval'] = np.std([r[f'cbm_true_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['mean_cbm_true_accuracy_test'] = np.mean([r[f'cbm_true_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)
r['std_cbm_true_accuracy_test'] = np.std([r[f'cbm_true_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)

r['mean_distiller_true_accuracy_trainval'] = np.mean([r[f'distiller_true_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['std_distiller_true_accuracy_trainval'] = np.std([r[f'distiller_true_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['mean_distiller_true_accuracy_test'] = np.mean([r[f'distiller_true_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)
r['std_distiller_true_accuracy_test'] = np.std([r[f'distiller_true_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)

r['mean_distiller_cbm_accuracy_trainval'] = np.mean([r[f'distiller_cbm_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['std_distiller_cbm_accuracy_trainval'] = np.std([r[f'distiller_cbm_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['mean_distiller_cbm_accuracy_test'] = np.mean([r[f'distiller_cbm_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)
r['std_distiller_cbm_accuracy_test'] = np.std([r[f'distiller_cbm_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)

#correct overlap

r['mean_%_correct_overlap_trainval'] = np.mean([r[f'%_correct_seed{seed}_overlap_trainval'] for seed in range(1, 4)], axis = 0)
r['std_%_correct_overlap_trainval'] = np.std([r[f'%_correct_seed{seed}_overlap_trainval'] for seed in range(1, 4)], axis = 0)
r['mean_%_correct_overlap_test'] = np.mean([r[f'%_correct_seed{seed}_overlap_test'] for seed in range(1, 4)], axis = 0)
r['std_%_correct_overlap_test'] = np.std([r[f'%_correct_seed{seed}_overlap_test'] for seed in range(1, 4)], axis = 0)

#edited performance

r['mean_edited_cbm_true_accuracy_trainval'] = np.mean([r[f'edited_cbm_true_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['std_edited_cbm_true_accuracy_trainval'] = np.std([r[f'edited_cbm_true_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['mean_edited_cbm_true_accuracy_test'] = np.mean([r[f'edited_cbm_true_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)
r['std_edited_cbm_true_accuracy_test'] = np.std([r[f'edited_cbm_true_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)

r['mean_edited_distiller_true_accuracy_trainval'] = np.mean([r[f'edited_distiller_true_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['std_edited_distiller_true_accuracy_trainval'] = np.std([r[f'edited_distiller_true_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['mean_edited_distiller_true_accuracy_test'] = np.mean([r[f'edited_distiller_true_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)
r['std_edited_distiller_true_accuracy_test'] = np.std([r[f'edited_distiller_true_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)

r['mean_edited_distiller_cbm_accuracy_trainval'] = np.mean([r[f'edited_distiller_cbm_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['std_edited_distiller_cbm_accuracy_trainval'] = np.std([r[f'edited_distiller_cbm_seed{seed}_accuracy_trainval'] for seed in range(1, 4)], axis = 0)
r['mean_edited_distiller_cbm_accuracy_test'] = np.mean([r[f'edited_distiller_cbm_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)
r['std_edited_distiller_cbm_accuracy_test'] = np.std([r[f'edited_distiller_cbm_seed{seed}_accuracy_test'] for seed in range(1, 4)], axis = 0)

#edited correct overlap

r['mean_edited_%_correct_overlap_trainval'] = np.mean([r[f'edited_%_correct_seed{seed}_overlap_trainval'] for seed in range(1, 4)], axis = 0)
r['std_edited_%_correct_overlap_trainval'] = np.std([r[f'edited_%_correct_seed{seed}_overlap_trainval'] for seed in range(1, 4)], axis = 0)
r['mean_edited_%_correct_overlap_test'] = np.mean([r[f'edited_%_correct_seed{seed}_overlap_test'] for seed in range(1, 4)], axis = 0)
r['std_edited_%_correct_overlap_test'] = np.std([r[f'edited_%_correct_seed{seed}_overlap_test'] for seed in range(1, 4)], axis = 0)

In [29]:
r = r.drop(columns  = ['save_dir', 'save_dir_unique', 'use_cache', 'device']
      +['cbm_true_seed1_accuracy_trainval', 'cbm_true_seed1_accuracy_test',
       'distiller_true_seed1_accuracy_trainval',
       'distiller_true_seed1_accuracy_test',
       'distiller_cbm_seed1_accuracy_trainval',
       'distiller_cbm_seed1_accuracy_test', '%_correct_seed1_overlap_trainval',
       '%_correct_seed1_overlap_test',
       'edited_cbm_true_seed1_accuracy_trainval',
       'edited_cbm_true_seed1_accuracy_test',
       'edited_distiller_true_seed1_accuracy_trainval',
       'edited_distiller_true_seed1_accuracy_test',
       'edited_distiller_cbm_seed1_accuracy_trainval',
       'edited_distiller_cbm_seed1_accuracy_test',
       'edited_%_correct_seed1_overlap_trainval',
       'edited_%_correct_seed1_overlap_test',
       'cbm_true_seed2_accuracy_trainval', 'cbm_true_seed2_accuracy_test',
       'distiller_true_seed2_accuracy_trainval',
       'distiller_true_seed2_accuracy_test',
       'distiller_cbm_seed2_accuracy_trainval',
       'distiller_cbm_seed2_accuracy_test', '%_correct_seed2_overlap_trainval',
       '%_correct_seed2_overlap_test',
       'edited_cbm_true_seed2_accuracy_trainval',
       'edited_cbm_true_seed2_accuracy_test',
       'edited_distiller_true_seed2_accuracy_trainval',
       'edited_distiller_true_seed2_accuracy_test',
       'edited_distiller_cbm_seed2_accuracy_trainval',
       'edited_distiller_cbm_seed2_accuracy_test',
       'edited_%_correct_seed2_overlap_trainval',
       'edited_%_correct_seed2_overlap_test',
       'cbm_true_seed3_accuracy_trainval', 'cbm_true_seed3_accuracy_test',
       'distiller_true_seed3_accuracy_trainval',
       'distiller_true_seed3_accuracy_test',
       'distiller_cbm_seed3_accuracy_trainval',
       'distiller_cbm_seed3_accuracy_test', '%_correct_seed3_overlap_trainval',
       '%_correct_seed3_overlap_test',
       'edited_cbm_true_seed3_accuracy_trainval',
       'edited_cbm_true_seed3_accuracy_test',
       'edited_distiller_true_seed3_accuracy_trainval',
       'edited_distiller_true_seed3_accuracy_tesst',
       'edited_distiller_cbm_seed3_accuracy_trainval',
       'edited_distiller_cbm_seed3_accuracy_test',
       'edited_%_correct_seed3_overlap_trainval',
       'edited_%_correct_seed3_overlap_test'])

In [77]:
r.columns

Index(['task_type', 'model_name', 'X_type', 'thresh', 'Y_type', 'max_rules',
       'max_trees', 'max_depth', 'pre_interaction', 'pre_max_features',
       'post_interaction', 'post_max_features', 'mo', 'concepts_to_edit',
       'mean_cbm_true_accuracy_trainval', 'std_cbm_true_accuracy_trainval',
       'mean_cbm_true_accuracy_test', 'std_cbm_true_accuracy_test',
       'mean_distiller_true_accuracy_trainval',
       'std_distiller_true_accuracy_trainval',
       'mean_distiller_true_accuracy_test', 'std_distiller_true_accuracy_test',
       'mean_distiller_cbm_accuracy_trainval',
       'std_distiller_cbm_accuracy_trainval',
       'mean_distiller_cbm_accuracy_test', 'std_distiller_cbm_accuracy_test',
       'mean_%_correct_overlap_trainval', 'std_%_correct_overlap_trainval',
       'mean_%_correct_overlap_test', 'std_%_correct_overlap_test',
       'mean_edited_cbm_true_accuracy_trainval',
       'std_edited_cbm_true_accuracy_trainval',
       'mean_edited_cbm_true_accuracy_test',
 

In [83]:
r[r.model_name =='FTDHydraRegressorCV'].sort_values('mean_distiller_true_accuracy_test', ascending=False)[['model_name', 'post_max_features']]

,model_name,post_max_features
16,FTDHydraRegressorCV,7.0
33,FTDHydraRegressorCV,7.0
3,FTDHydraRegressorCV,7.0
39,FTDHydraRegressorCV,7.0
20,FTDHydraRegressorCV,7.0
21,FTDHydraRegressorCV,7.0
8,FTDHydraRegressorCV,7.0


In [42]:
r[r['X_type'] == 'cluster']['mean_distiller_cbm_accuracy_test']

5     0.771258
6     0.789840
22    0.771258
23    0.790818
26    0.790128
28    0.769187
32    0.790818
35    0.769187
Name: mean_distiller_cbm_accuracy_test, dtype: float64

In [53]:
r[r['X_type'] != 'cluster']['mean_distiller_true_accuracy_trainval']

0     0.966355
1     0.974320
2     0.816065
3     0.899454
4     0.971201
7     0.971201
8     0.890430
9     0.981061
10    0.985071
11    0.983066
12    0.966355
13    0.981061
14    0.772727
15    0.983066
16    0.938336
17    0.773674
18    0.981172
19    0.699978
20    0.926638
21    0.888703
24    0.985071
25    0.976381
27    0.974320
29    0.826705
30    0.973039
31    0.981172
33    0.894942
34    0.762199
36    0.976381
37    0.699086
38    0.773117
39    0.890374
Name: mean_distiller_true_accuracy_trainval, dtype: float64

In [50]:
r[r['X_type'] != 'cluster']['mean_cbm_true_accuracy_test']

0     0.777126
1     0.777126
2     0.777126
3     0.777126
4     0.777126
7     0.777126
8     0.777126
9     0.777126
10    0.777126
11    0.777126
12    0.777126
13    0.777126
14    0.777126
15    0.777126
16    0.777126
17    0.777126
18    0.777126
19    0.777126
20    0.777126
21    0.777126
24    0.777126
25    0.777126
27    0.777126
29    0.777126
30    0.777126
31    0.777126
33    0.777126
34    0.777126
36    0.777126
37    0.777126
38    0.777126
39    0.777126
Name: mean_cbm_true_accuracy_test, dtype: float64

In [25]:
r[r['model_name']=='FTDHydraRegressorCV'].sort_values('mean_edited_distiller_true_accuracy_test', ascending=False).mean_edited_distiller_true_accuracy_test #['mean_edited_distiller_true_accuracy_test'] #['mean_distiller_true_accuracy_test']

3     0.709527
14    0.685997
31    0.683983
26    0.682890
18    0.638879
19    0.624957
6     0.621160
Name: mean_edited_distiller_true_accuracy_test, dtype: float64

In [ ]:
r[['model_name','mean_distiller_cbm_accuracy_trainval', 'mean_edited_distiller_cbm_accuracy_trainval', 'mean_distiller_cbm_accuracy_test', 'mean_edited_distiller_cbm_accuracy_test']]

In [ ]:
r[['model_name','mean_cbm_true_accuracy_trainval', 'mean_edited_cbm_true_accuracy_trainval', 'mean_distiller_true_accuracy_trainval', 'mean_edited_distiller_true_accuracy_trainval',]]

In [ ]:
r[['model_name', 'mean_%_correct_overlap_trainval', 'mean_%_correct_overlap_test', 'mean_edited_%_correct_overlap_trainval', 'mean_edited_%_correct_overlap_test']]